In [40]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from ipywidgets import interact, FloatSlider
from scipy.stats import norm

# Normal Inverse Gaussian Model 

$$ X(t; \alpha, \beta, \delta, \sigma) = \beta \Gamma(t; \delta) + \sigma W(\Gamma(t; \delta))$$
- $\Gamma(t;\delta)$: Gamma process with shape parameter t and scale parameter $\delta$
- $W(\Gamma(t; \delta))$: Brownian motion evaluated at gamma process
- $\alpha$ > 0
- $|\beta| < \alpha $


In [41]:
@dataclass
class NIGparams:
    alpha : float
    beta : float
    delta: float
    sigma: float


In [42]:
def NIG_cf_St(u, S0, T, r, q, p:NIGparams):
    omega = p.delta * (np.sqrt(p.alpha ** 2 - (p.beta  + 1)**2) - np.sqrt(p.alpha ** 2 - p.beta ** 2))
    first_term = np.exp(1j * u * (r - q + omega - 0.5 * p.sigma ** 2) * T)
    phi_x = np.exp(p.delta * T * (np.sqrt(p.alpha ** 2 - p.beta ** 2) - np.sqrt(p.alpha ** 2 - (p.beta  + 1j * u) ** 2)) - 0.5 * p.sigma ** 2 * u ** 2 * T)
    return first_term * phi_x

def NIG_cf_log_St(u, S0, T, r, q, p:NIGparams):
    term = np.exp(1j * u * np.log(S0))
    omega = p.delta * (np.sqrt(p.alpha ** 2 - (p.beta  + 1)**2) - np.sqrt(p.alpha ** 2 - p.beta ** 2))
    first_term = np.exp(1j * u * (r - q + omega - 0.5 * p.sigma ** 2) * T)
    phi_x = np.exp(p.delta * T * (np.sqrt(p.alpha ** 2 - p.beta ** 2) - np.sqrt(p.alpha ** 2 - (p.beta  + 1j * u) ** 2)) - 0.5 * p.sigma ** 2 * u ** 2 * T)
    return term * first_term * phi_x



def cos_price_call(S0, K, T, r, q, p:NIGparams, N=2**15, L = 15):
    a = 0 - L * np.sqrt(T)
    b = 0 + L * np.sqrt(T)
    x0 = np.log(S0/K)
    k = np.arange(N)                      # 0,1,...,N-1
    u = k * np.pi / (b - a)               # frequencies

   


    # Compute Vk coefficients analytically

    c = 0
    d = b                                

    

    # psi_k(c,d)
    psi = np.zeros(N)
    psi[0] = d - c
    psi[1:] = (np.sin(k[1:] * np.pi * (d - a)/(b - a)) - np.sin(k[1:] * np.pi * (c - a)/(b - a))) * (b - a) / (k[1:] * np.pi)

    # chi_k(c,d)
    term1 = 1.0 / (1.0 + (k * np.pi / (b - a)) ** 2)
    term2 = np.cos(k * np.pi  * ((d - a)/ (b - a))) * np.exp(d)
    term3 = np.cos(k * np.pi  * ((c - a)/ (b - a))) * np.exp(c)
    term4 = (k * np.pi / (b - a)) * np.sin(k * np.pi * ((d - a)/(b - a))) * np.exp(d)
    term5 = (k * np.pi / (b - a)) * np.sin(k * np.pi * ((c - a)/(b - a))) * np.exp(c)
    chi = term1 * (term2  - term3 + term4 - term5)

    Vk = 2.0 / (b - a) * (chi - psi)
    Vk[0] *= 0.5  # first term has weight 1/2 in COS expansion

    # Characteristic function part
    temp = (NIG_cf_St(u, S0, T, r , q, p) * Vk)
    mat = np.exp(1j * (x0 - a) * u)
    value = np.exp(-r * T) * K * np.real(mat @ temp)
    # COS formula
    
    return float(value)

hp = NIGparams(alpha = 2.0, beta = 0.7, delta = 0.1, sigma = 0.02)
call_price = cos_price_call(100.0, 100.0, 0.06, 0.02, 0.0, hp)
print(call_price)

1.1026854641153505


In [43]:
def European_Call(S0, K, T, r , q, sigma):
    d1 = (np.log(S0/K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return  S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_vega(S0, K, T, r, q, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * norm.pdf(d1) * np.sqrt(T)

def get_implied_vol(market_price, S0, K, T, r, q, max_iter = 200, tolerance = 1e-05):
    sigma = 0.3
    for _ in range(max_iter):
        bs_price = European_Call(S0, K, T, r, q, sigma)
        diff = bs_price - market_price
        if abs(diff) < tolerance:
            return sigma
        vega = bs_vega(S0,K,T,r, q, sigma)
        if vega == 0:
            return None
        sigma -= diff/vega
    return None

In [ ]:

def plot_BS_implied_vol(S0, T, r, q, alpha, beta, delta, sigma):
    K_min = S0 * 0.90
    K_max = S0 * 1.05
    fig, ax = plt.subplots(1,2, figsize = (12,6))
    
    hp = NIGparams(alpha = alpha, beta = beta, delta = delta, sigma = sigma)
    K_grid = np.linspace(K_min, K_max, 300)
   
    
    nig_prices = [cos_price_call(S0, k, T, r , q, hp) for k in K_grid]
   
    BS_implied_vols = [get_implied_vol(nig_price, S0, strike, T, r, q) for strike, nig_price in zip(K_grid,nig_prices)] 
   
    ax[0].plot(K_grid, BS_implied_vols, color = 'black', label = "COS")
 
    ax[0].set_xlabel("Strike")
    ax[0].set_ylabel("BS implied volatility (%)")
    ax[0].set_title("BS implied volatility(%) for NIG model")
    ax[0].grid(True)
    ax[0].legend()
    ax[1].plot(K_grid, nig_prices, color = 'black', label = "NIG model")
    ax[1].set_xlabel("Strike")
    ax[1].set_ylabel("Call option price")
    ax[1].set_title("Call option price for NIG model")
    ax[1].grid(True)
    ax[1].legend()
    plt.show()


interact(plot_BS_implied_vol,
        s_min = FloatSlider(value=60, min=20,  max=100, step=1,   description="S_min"),
         s_max = FloatSlider(value=160, min=100, max=300, step=1 , description="S_max"),
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=0.001, min=0.01, max=5.0, step=0.001, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         alpha=FloatSlider(value=7.99, min=0.0, max=10.0, step=0.005, description="alpha"),
         beta=FloatSlider(value=-3.69, min=-10.0, max=10.0, step=0.005, description="beta"),
         delta=FloatSlider(value=0.0763, min=0.00, max=0.99, step=0.005, description="delta"),
         sigma=FloatSlider(value=0.1467, min=0.001, max=5.0, step=0.005, description="sigma"),
        )
       

interactive(children=(FloatSlider(value=100.0, description='S0', max=300.0, min=20.0, step=1.0), FloatSlider(v…

<function __main__.plot_BS_implied_vol(S0, T, r, q, alpha, beta, delta, sigma)>

In [ ]:
def risk_neutral_density_log_St(x_array, S0, r, q, T, alpha, beta, delta, sigma,
                                u_max=200, n_steps=5000):
    """
    Returns f_{X_T}(x) for X_T = log S_T at points x_array.
    """
    hp = NIGparams(alpha = alpha, beta = beta, delta = delta, sigma = sigma)

    # integration grid in u
    u = np.linspace(0.0, u_max, n_steps)              # shape (N_u,)
    phi = NIG_cf_log_St(u, S0, T, r, q, hp)               # shape (N_u,)


    # shape (N_u, N_x): outer product in exponent
    exp_term = np.exp(-1j * np.outer(u, x_array))
    integrand = np.real(exp_term * phi[:, None])      # broadcast phi over columns

    # integrate over u (axis=0) to get f_X(x) for each x
    fx = (1/np.pi) * np.trapezoid(integrand, u, axis=0)
    return fx   # shape (N_x,)


def plot_risk_neutral_density_St(s_min, s_max, S0, r, q, T, alpha, beta, delta, sigma,
                                 u_max=200, n_steps=5000):
    """
    Plots f_{S_T}(s) and returns the density values.
    """
    
    s_array = np.linspace(s_min, s_max, 200)
    x_array = np.log(s_array)

    # f_X(x)
    fx = risk_neutral_density_log_St(x_array, S0, r, q, T, alpha, beta, delta, sigma, u_max=u_max, n_steps=n_steps)

    fS = fx / s_array

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(s_array, fS, color = 'black')
    ax.axvline(x=S0, color='red', linestyle='--', linewidth=2)
    ax.set_xlabel(r"$s$")
    ax.set_ylabel(r"$f_{S_T}(s)$")
    ax.set_title("Risk-neutral density of $S_T$ under VGB model")
    ax.grid(True)
    plt.show()

interact(plot_risk_neutral_density_St,
         s_min = FloatSlider(value=60, min=20,  max=100, step=1,   description="S_min"),
         s_max = FloatSlider(value=160, min=100, max=300, step=1 , description="S_max"),
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=0.002, min=0.01, max=5.0, step=0.001, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         q=FloatSlider(value=0.0, min=0.0, max=0.1, step=0.002, description="q"),
         alpha=FloatSlider(value=7.99, min=0.0, max=10.0, step=0.005, description="alpha"),
         beta=FloatSlider(value=-3.69, min=-10.0, max=10.0, step=0.005, description="beta"),
         delta=FloatSlider(value=0.0763, min=0.00, max=0.99, step=0.005, description="delta"),
         sigma=FloatSlider(value=0.1467, min=0.001, max=5.0, step=0.005, description="sigma"),
        )

        


interactive(children=(FloatSlider(value=60.0, description='S_min', min=20.0, step=1.0), FloatSlider(value=160.…

<function __main__.plot_risk_neutral_density_St(s_min, s_max, S0, r, q, T, alpha, beta, delta, sigma, u_max=200, n_steps=5000)>